In [1]:
import pandas as pd 

# Generate Sensors85

In [2]:
df = pd.read_pickle('../datasets/preprocessed/sensors.pkl')
df_feats = pd.read_pickle('../datasets/preprocessed/sensors_feats.pkl')

pats=set(df.groupby('patient').trial.count().sort_values(ascending=False).index[:85])

df = df[df.patient.isin(pats)].reset_index(drop=True)
df_feats = df_feats[df_feats.patient.isin(pats)].reset_index(drop=True)

In [5]:
df.to_pickle('../datasets/preprocessed/sensors85.pkl')
df_feats.to_pickle('../datasets/preprocessed/sensors85_feats.pkl')

# Generate mix split of all dataset

In [140]:
import sys, os
sys.path.append('../code/process/')
from data_splitting import main as split
from omegaconf import OmegaConf 
import joblib
sys.path.append('../code/train/')
from core.utils import get_nested_fold_idx
import numpy as np

def compute_mix_stats(conf):
    all_split_df = joblib.load(conf.path.split_df_path)
    samp_shared = []
    sub_shared = []
    for foldIdx, (folds_train, folds_val, folds_test) in enumerate(get_nested_fold_idx(len(all_split_df))):
        if (conf.param_split.type=='hoo') and (foldIdx==1):  break
        train_df = pd.concat(np.array(all_split_df)[folds_train])
        val_df = pd.concat(np.array(all_split_df)[folds_val])
        test_df = pd.concat(np.array(all_split_df)[folds_test])

        samp_shared.append(test_df.patient.isin(train_df.patient).mean())
        sub_shared.append(np.mean([sub in train_df.patient.unique() for sub in test_df.patient.unique()]))

    print(samp_shared)
    print(' - samp_shared',np.mean(samp_shared))
    print(sub_shared)
    print(' - sub_shared',np.mean(sub_shared))
    return samp_shared, sub_shared


In [9]:
path_conf = '../code/process/core/config/splitting/'
processed_df_path = '../datasets/preprocessed/'
split_df_path = '../datasets/mix/'

In [136]:
for db in ['sensors', 'bcg', 'ppgbp', 'uci']:
    print(db)
    conf = OmegaConf.load(path_conf+db+'_split.yaml')
    conf.param_split.is_mix = True
    name_db = 'sensors85' if 'sensors'==db else db
    if db=='uci':
        conf.param_split.frac=0.25
    
    conf.path.processed_df_path = processed_df_path+name_db+'.pkl'
    conf.path.split_df_path = split_df_path+name_db+'.pkl'
    print(conf.param_split.is_mix, conf.path.processed_df_path, conf.path.split_df_path)
    split(conf)
    
    conf.path.processed_df_path = processed_df_path+name_db+'_feats.pkl'
    conf.path.split_df_path = split_df_path+name_db+'_feats.pkl'
    print(conf.param_split.is_mix, conf.path.processed_df_path, conf.path.split_df_path)
    split(conf)
    
    compute_mix_stats(conf)
    
    print()
    print()
    print()

sensors
True ../datasets/preprocessed/sensors85.pkl ../datasets/mix/sensors85.pkl


/root/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[2, 3, 4] [1] [0]
[0, 3, 4] [2] [1]
[0, 1, 4] [3] [2]
[0, 1, 2] [4] [3]
[1, 2, 3] [0] [4]
SBP: MAE 18.4777 | ME 0.0001 | STD 22.6073
DBP: MAE 9.2626 | ME 0.0000 | STD 11.6467
total amount 3824
True ../datasets/preprocessed/sensors85_feats.pkl ../datasets/mix/sensors85_feats.pkl


/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_df = pd.concat(np.array(all_split_df)[folds_train])
/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_df = pd.concat(np.array(all_split_df)[folds_val])
/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or

[2, 3, 4] [1] [0]
[0, 3, 4] [2] [1]


...

SBP: MAE 17.9078 | ME 0.0910 | STD 22.0238
DBP: MAE 8.5851 | ME 0.0418 | STD 11.1129
total amount 410596
True ../datasets/preprocessed/UCI_feats.pkl ../datasets/mix/UCI_feats.pkl
[2] [1] [0]


/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_df = pd.concat(np.array(all_split_df)[folds_train])
/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_df = pd.concat(np.array(all_split_df)[folds_val])
/home/jovyan/BP/bp-algorithm/analysis/../code/process/data_splitting.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or

SBP: MAE 17.9078 | ME 0.0910 | STD 22.0238
DBP: MAE 8.5851 | ME 0.0418 | STD 11.1129
total amount 410596


/tmp/ipykernel_7702/679163948.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_df = pd.concat(np.array(all_split_df)[folds_train])
/tmp/ipykernel_7702/679163948.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_df = pd.concat(np.array(all_split_df)[folds_val])
/tmp/ipykernel_7702/679163948.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when

[0.9954505158355171, 0.9866990530868848, 0.9807239348804988]
 - samp_shared 0.9876245012676336
[0.9614968635085442, 0.9189707707933642, 0.8384283754654125]
 - sub_shared 0.9062986699224403





In [150]:
compute_mix_stats(conf)

/tmp/ipykernel_7702/146089765.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_df = pd.concat(np.array(all_split_df)[folds_train])
/tmp/ipykernel_7702/146089765.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_df = pd.concat(np.array(all_split_df)[folds_val])
/tmp/ipykernel_7702/146089765.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when

[0.9954505158355171]
 - samp_shared 0.9954505158355171
[0.9614968635085442]
 - sub_shared 0.9614968635085442


([0.9954505158355171], [0.9614968635085442])

In [167]:
for db in ['sensors85', 'bcg', 'ppgbp', 'uci']:
    for end in ['','_feats']:
        filename = f'../datasets/mix/{db}{end}.pkl'
        all_split_df = joblib.load(filename)
        for i in range(len(all_split_df)):
            if 'agg_ohe' in all_split_df[i].columns:
                all_split_df[i].drop(columns=['agg_ohe'], inplace=True)
        joblib.dump(all_split_df, filename)